In [ ]:
import requests
import json
import sqlite3
import pandas as pd
import tokens.credential as credential
import time

from pathlib import Path
from utils import *

MY_ACCESS_TOKEN = credential.get_token()

In [ ]:
# 종목정보 리스트
def fn_ka10099(token, data, cont_yn='N', next_key=''):
	# 1. 요청할 API URL
	host = 'https://mockapi.kiwoom.com' # 모의투자
	endpoint = '/api/dostk/stkinfo'
	url =  host + endpoint

	# 2. header 데이터
	headers = {
		'Content-Type': 'application/json;charset=UTF-8', # 컨텐츠타입
		'authorization': f'Bearer {token}', # 접근토큰
		'cont-yn': cont_yn, # 연속조회여부
		'next-key': next_key, # 연속조회키
		'api-id': 'ka10099', # TR명
	}

	# 3. http POST 요청
	response = requests.post(url, headers=headers, json=data)

	# 4. 응답 상태 코드와 데이터 출력
	print('Code:', response.status_code)
	print('Header:', json.dumps({key: response.headers.get(key) for key in ['next-key', 'cont-yn', 'api-id']}, indent=4, ensure_ascii=False))
	return response.json()["list"]

# 실행 구간
if __name__ == '__main__':

	# 2. 요청 데이터
	params = {
		'mrkt_tp': '0', # 시장구분 0:코스피,10:코스닥,3:ELW,8:ETF,30:K-OTC,50:코넥스,5:신주인수권,4:뮤추얼펀드,6:리츠,9:하이일드
	}

	# 3. API 실행
	data = fn_ka10099(token=MY_ACCESS_TOKEN, data=params)

	# next-key, cont-yn 값이 있을 경우
	# fn_ka10099(token=MY_ACCESS_TOKEN, data=params, cont_yn='Y', next_key='nextkey..')

    with sqlite3.connect("market.db") as conn:
    conn.execute("""
    CREATE TABLE IF NOT EXISTS stock_master (
    code TEXT PRIMARY KEY,          -- 종목코드 (000020)
    name TEXT NOT NULL,             -- 종목명
    listCount TEXT,                 -- 상장주식수 (자리수가 커서 TEXT 권장)
    auditInfo TEXT,                 -- 상태 (정상 등)
    regDay TEXT,                    -- 상장일 (YYYYMMDD)
    lastPrice INTEGER,              -- 현재가
    state TEXT,                     -- 증거금 / 신용 정보
    marketCode INTEGER,             -- 시장 코드 (0 등)
    marketName TEXT,                -- 거래소 / 코스닥
    upName TEXT,                    -- 업종
    upSizeName TEXT,                -- 대/중/소형
    companyClassName TEXT,          -- (빈 값 존재)
    orderWarning INTEGER,           -- 0/1
    nxtEnable TEXT,                 -- Y / N
    kind TEXT                        -- A 등
    )
    """)
    conn.commit()

df = pd.DataFrame(data)

with sqlite3.connect("market.db") as conn:
    df.to_sql("stock_mas", conn, if_exists="append", index=False)

    conn.execute("""
    INSERT OR IGNORE INTO stock_master (code, name, listCount, auditInfo, regDay, lastPrice, state, marketCode, marketName, upName, upSizeName, companyClassName, orderWarning, nxtEnable, kind)
    SELECT code, name, listCount, auditInfo, regDay, lastPrice, state, marketCode, marketName, upName, upSizeName, companyClassName, orderWarning, nxtEnable, kind
    FROM stock_mas;
    """)

    conn.execute("DROP TABLE stock_mas;")

In [ ]:
# 일별주가요청
def fn_ka10086(token, data, cont_yn='N', next_key=''):
    # 1. 요청할 API URL
    host = 'https://mockapi.kiwoom.com' # 모의투자
    endpoint = '/api/dostk/mrkcond'
    url =  host + endpoint

    # 2. header 데이터
    headers = {
        'authorization': f'Bearer {token}', # 접근토큰
        'cont-yn': cont_yn, # 연속조회여부
        'next-key': next_key, # 연속조회키
        'api-id': 'ka10086', # TR명
    }
    # 3. http POST 요청
    response = requests.post(url, headers=headers, json=data)

    return response
    # # 4. 응답 상태 코드와 데이터 출력
    # print('Code:', response.status_code)
    # print('Header:', json.dumps({key: response.headers.get(key) for key in ['next-key', 'cont-yn', 'api-id']}, indent=4, ensure_ascii=False))
    # return response

In [ ]:
with sqlite3.connect("market.db") as conn:
    code_df = pd.read_sql("SELECT code, name FROM stock_master WHERE marketCode=0", conn)

In [ ]:
# 2. 요청 데이터
MAX_RETRY=3
qry_dt = pd.Timestamp.today()
for code in code_df["code"]:
    stk_cd = code 
    records = []
    cont_yn="N"
    next_key=""

    params = {
    	'stk_cd': stk_cd, # 종목코드 거래소별 종목코드 (KRX:039490,NXT:039490_NX,SOR:039490_AL)
    	'qry_dt': qry_dt.strftime("%Y%m%d"), # 조회일자 YYYYMMDD
    	'indc_tp': '0', # 표시구분 0:수량, 1:금액(백만원)
    }
    # 3. API 실행
    for i in range(10):
        params['qry_dt'] = (qry_dt - pd.offsets.BDay(21*i)).strftime("%Y%m%d")
        response = fn_ka10086(token=MY_ACCESS_TOKEN, data=params, cont_yn=cont_yn, next_key=next_key)
        if response.status_code == 429:
            for _ in range(MAX_RETRY):
                time.sleep(1)
                response = fn_ka10086(token=MY_ACCESS_TOKEN, data=params, cont_yn=cont_yn, next_key=next_key)
                if response.status_code == 200: break

        elif response.status_code == 200:
            cont_yn = response.headers.get('cont_yn')
            next_key = response.headers.get('next_key')
            for row in response.json()['daly_stkpc']:
                records.append({
                "stk_cd": stk_cd,
                "date": row["date"],
                "open": to_int(row["open_pric"]),
                "high": to_int(row["high_pric"]),
                "low": to_int(row["low_pric"]),
                "close": to_int(row["close_pric"]),
                "volume": to_int(row["trde_qty"]),
            })
            if cont_yn=="N": break
                
        else:
            print(response.status_code)
            
    df = pd.DataFrame(records)
    df = df.sort_values("date").reset_index(drop=True)

    with sqlite3.connect("market.db") as conn:
        conn.execute("""
    CREATE TABLE IF NOT EXISTS daily_price (
        stk_cd  TEXT NOT NULL,
        date    DATE NOT NULL,
        open    INTEGER NOT NULL,
        high    INTEGER NOT NULL,
        low     INTEGER NOT NULL,
        close   INTEGER NOT NULL,
        volume  INTEGER NOT NULL,
        PRIMARY KEY (stk_cd, date)
    )
    """)
        conn.commit()

    with sqlite3.connect("market.db") as conn:
        df.to_sql("tmp_daily_price", conn, if_exists="replace", index=False)

        conn.execute("""
    INSERT OR REPLACE INTO daily_price (stk_cd, "date", open, high, low, close, volume)
    SELECT stk_cd, "date", open, high, low, close, volume
    FROM tmp_daily_price;
    """)

        conn.execute("DROP TABLE tmp_daily_price;")


In [ ]:
df

In [ ]:
df.style.format(thousands=',')

In [ ]:
# 주식 매수주문
def fn_kt10000(token, data, cont_yn='N', next_key=''):
	# 1. 요청할 API URL
	host = 'https://mockapi.kiwoom.com' 
	endpoint = '/api/dostk/ordr'
	url =  host + endpoint

	# 2. header 데이터
	headers = {
		'Content-Type': 'application/json;charset=UTF-8', # 컨텐츠타입
		'authorization': f'Bearer {token}', # 접근토큰
		'cont-yn': cont_yn, # 연속조회여부
		'next-key': next_key, # 연속조회키
		'api-id': 'kt10000', # TR명
	}

	# 3. http POST 요청
	response = requests.post(url, headers=headers, json=data)

	# 4. 응답 상태 코드와 데이터 출력
	print('Code:', response.status_code)
	print('Header:', json.dumps({key: response.headers.get(key) for key in ['next-key', 'cont-yn', 'api-id']}, indent=4, ensure_ascii=False))
	print('Body:', json.dumps(response.json(), indent=4, ensure_ascii=False))  # JSON 응답을 파싱하여 출력

# 실행 구간
if __name__ == '__main__':
	# 1. 토큰 설정

	# 2. 요청 데이터
	params = {
		'dmst_stex_tp': 'KRX', # 국내거래소구분 KRX,NXT,SOR
		'stk_cd': '005930', # 종목코드 
		'ord_qty': '1', # 주문수량 
		'ord_uv': '', # 주문단가 
		'trde_tp': '3', # 매매구분 0:보통 , 3:시장가 , 5:조건부지정가 , 81:장마감후시간외 , 61:장시작전시간외, 62:시간외단일가 , 6:최유리지정가 , 7:최우선지정가 , 10:보통(IOC) , 13:시장가(IOC) , 16:최유리(IOC) , 20:보통(FOK) , 23:시장가(FOK) , 26:최유리(FOK) , 28:스톱지정가,29:중간가,30:중간가(IOC),31:중간가(FOK)
		'cond_uv': '', # 조건단가 
	}

	# 3. API 실행
	fn_kt10000(token=MY_ACCESS_TOKEN, data=params)

	# next-key, cont-yn 값이 있을 경우
	# fn_kt10000(token=MY_ACCESS_TOKEN, data=params, cont_yn='Y', next_key='nextkey..')